In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("kidney_disease_train.csv")
df.head(7)

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,157,62.0,70.0,1.025,3.0,0.0,normal,abnormal,notpresent,notpresent,...,39.0,7900,3.9,yes,yes,no,good,no,no,ckd
1,109,54.0,70.0,NaN,NaN,NaN,NaN,NaN,notpresent,notpresent,...,NaN,NaN,NaN,no,yes,no,good,no,no,ckd
2,17,47.0,80.0,NaN,NaN,NaN,NaN,NaN,notpresent,notpresent,...,NaN,NaN,NaN,yes,no,no,poor,no,no,ckd
3,347,43.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,...,43.0,7200,5.5,no,no,no,good,no,no,notckd
4,24,42.0,100.0,1.015,4.0,0.0,normal,abnormal,notpresent,present,...,39.0,8300,4.6,yes,no,no,poor,no,no,ckd
5,175,60.0,50.0,1.010,0.0,0.0,NaN,normal,notpresent,notpresent,...,NaN,4200,3.4,yes,no,no,good,no,no,ckd
6,351,29.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,...,40.0,9900,4.7,no,no,no,good,no,no,notckd


In [ ]:
columns_to_retain = ['sg','al','sc','hemo','pcv','wbcc','rbcc','htn','classification']

df = df.drop([col for col in df.columns if not col in columns_to_retain],axis =1)

df = df.dropna(axis=0)

In [ ]:
for column in df.columns:
    if df[column].dtype == np.number:
        continue
    df[column] = LabelEncoder().fit_transform( df[column])

<ipython-input-5-ee409e124a11>:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:
<ipython-input-5-ee409e124a11>:2: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if df[column].dtype == np.number:


In [ ]:
X = df.drop(['classification'],axis=1)
Y = df['classification']

In [ ]:
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names] = x_scaler.transform(X)
X[column_names]

,sg,al,sc,hemo,pcv,htn
0,1.00,0.6,0.055085,0.646259,0.666667,1.0
3,1.00,0.0,0.025424,1.000000,0.755556,0.0
4,0.50,0.8,0.042373,0.544218,0.666667,1.0
6,0.75,0.0,0.021186,0.979592,0.688889,0.0
7,1.00,0.0,0.025424,0.829932,0.777778,0.0
...,...,...,...,...,...,...
273,0.50,0.4,0.148305,0.312925,0.333333,1.0
275,0.25,0.2,0.122881,0.455782,0.422222,1.0
277,1.00,0.0,0.029661,0.761905,0.711111,0.0
278,0.75,0.0,0.004237,0.714286,0.777778,0.0


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,shuffle=True)

In [ ]:
def models(X_train,Y_train):


    from sklearn.linear_model import LogisticRegression
    log = LogisticRegression(random_state=0)
    log.fit(X_train,Y_train)



    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier(criterion = "entropy" , random_state=0)
    tree.fit(X_train,Y_train)


    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators = 10 , criterion = "entropy",random_state=0)
    forest.fit(X_train,Y_train)


    print('[0] Logistic Regression Training Accuracy : ',log.score(X_train,Y_train))
    print('[1] Decision Tree Classifier Training Accuracy : ',tree.score(X_train,Y_train))
    print('[2] Random Forest Classifier Training Accuracy : ',forest.score(X_train,Y_train))

    return log,tree,forest


In [ ]:
model = models(X_train,Y_train)

[0] Logistic Regression Training Accuracy :  0.9551282051282052
[1] Decision Tree Classifier Training Accuracy :  1.0
[2] Random Forest Classifier Training Accuracy :  1.0


In [ ]:
from sklearn.metrics import confusion_matrix

for i in range(len(model)):
    print("Model ", i)

    cm = confusion_matrix(Y_test,model[i].predict(X_test))
    # [[true_negative , false_postive] [false_negative,true_positive]]
    TP = cm[1][1]
    TN = cm[0][0]
    FP = cm[0][1]
    FN = cm[1][0]

    print(cm)
    print("Testing Accuracy =  ",(TP + TN)/(TP + TN + FN + FP))
    print()

Model  0
[[19  1]
 [ 0 20]]
Testing Accuracy =   0.975

Model  1
[[19  1]
 [ 0 20]]
Testing Accuracy =   0.975

Model  2
[[20  0]
 [ 0 20]]
Testing Accuracy =   1.0



In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


for i in range(len(model)):
    print("Model ", i)
    print(classification_report(Y_test,model[i].predict(X_test)))
    print(accuracy_score(Y_test,model[i].predict(X_test)))
    print()

Model  0
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       0.95      1.00      0.98        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40

0.975

Model  1
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       0.95      1.00      0.98        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40

0.975

Model  2
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        20
           1       1.00      1.00      1.00        20

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00

In [ ]:
pred = model[1].predict(X_test)
print(pred)
print()
print(Y_test)

[0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1
 1 1 0]

225    0
240    0
262    0
179    1
182    1
186    0
159    1
207    1
252    1
219    0
237    1
254    1
204    1
261    1
34     0
119    1
164    0
146    1
89     0
108    0
194    1
161    1
7      1
224    1
187    0
200    0
20     0
246    0
4      0
25     1
154    0
16     0
157    0
124    0
27     0
19     1
222    1
42     1
156    1
122    0
Name: classification, dtype: int64


In [ ]:
def  SaveModel():
    import pickle
    with open("CKD_Model","wb") as f:
        pickle.dump(model[1],f)

In [ ]:
SaveModel()

In [ ]:
import pickle
with open("CKD_Model","rb") as f:
    randomForest = pickle.load(f)

pred = randomForest.predict(X_test)
print(pred)
print()
print(Y_test)

[0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1
 1 1 0]

225    0
240    0
262    0
179    1
182    1
186    0
159    1
207    1
252    1
219    0
237    1
254    1
204    1
261    1
34     0
119    1
164    0
146    1
89     0
108    0
194    1
161    1
7      1
224    1
187    0
200    0
20     0
246    0
4      0
25     1
154    0
16     0
157    0
124    0
27     0
19     1
222    1
42     1
156    1
122    0
Name: classification, dtype: int64
